## Setup

In [ ]:
%pip install datasets
%pip install transformers
%pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.7 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from transformers import pipeline, set_seed
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/CS\ 182\ Final\ Project/Phase \2

/content/drive/.shortcut-targets-by-id/1hzhdcGA40OipfzF0SRT7omKvzCSi0q4r/CS 182 Final Project/Phase 2


## Dataset

In [ ]:
dataset_gradeschool = load_dataset('qwedsacf/grade-school-math-instructions')
dataset_gradeschool = dataset_gradeschool['train'].select_columns(['INSTRUCTION', 'RESPONSE'])
dataset_gradeschool = dataset_gradeschool.train_test_split(test_size=0.2, seed=20)
train_gradeschool, val_gradeschool = dataset_gradeschool['train'], dataset_gradeschool['test']

val_gradeschool_humaneval = val_gradeschool.train_test_split(test_size=100, seed=20)['test']

In [ ]:
# example
print('INSTRUCTION: ', train_gradeschool[0]['INSTRUCTION'])
print('RESPONSE: ', train_gradeschool[0]['RESPONSE'])

## Fine-tuning

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM

In [ ]:
device = 'cuda'
model_id = 'model_results/checkpoint-15000'
model = AutoPeftModelForCausalLM.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2-xl')

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, device=device)
text = '9+5='
result = generator(text)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartFo

In [ ]:
result[0]['generated_text']

'9+5=11 = 14 = 17 = 19 = 27 = 33 = 38 = 46 = 53 = 58 = 61 = 68 = 74 = 80 = 85 = 91 = 96 = 98 = 101 = 106 = 110 = 114 = 118 = 119 = 124 = 126 = 129 = 134 = 135 = 137 = 140 = 144 = 146 = 148 = 153 = 155 = 158 = 162 = 164 = 168 = 171 = 173 = 175 = 177 = 179 = 183 = 1901 = 1907 = 2064 = 2097 = 2102 = 2108 = 2116 = 2122 = 2129 = 2132 = 2137 = 2138 - 2145 = 2154 - 2151 = 2154 - 2153 = 2157 - 2163 - 2184 = 2173 - 2179 - 2214 - 2224 = 22222 + 22236 - 22304 = 22482 - 22466 - 22508 - 2259 - 2277 - 22078 - 22087 - 22081 - 220'